Data Products (a.k.a. deploying an app)
---------------------------------------

This self-guided **Python 3** notebook will you take you from displaying a local html document web to deploying a web-app locally that uses a pre-trained machine learning model to predict on the class of new data and to display it.  At this point you'll be ready for the individual assignment.

For the pair this afternoon you'll be attempting to deploy a nice-looking web page on AWS.  But first - get comfortable with Flask this morning and occasional tweaking of HTML.  You'll need both skills this afternoon.

You'll be bouncing back-and-forth between this notebook and the command line a lot.  The `%%writefile` command in many of the cells writes the python or HTML file that need to be opened or executed from the command line.

_So here we go:_

Web pages are written in HTML (HyperText Markup Language) and this HTML is rendered in a browser.  You'll be seeing a lot of HTML in this notebook.  Go to this [link](http://www.w3schools.com/html/) at w3schools.com if the brief explanation of the HTML in the comments isn't sufficient.  

Execute the cell below to create the `example_html_only.html` file then go to terminal to render it in your browser.

In [ ]:
%%writefile example_html_only.html

<!DOCTYPE html>
<html>
    <head>
        <meta charset="utf-8">
        <title>Page Title</title>
    </head>
    <body>
    <!-- page content below -->
        <h1>My Page</h1>
        <p>
            All the things I want to say.  
        </p>
        <p style="color: purple; text-align: right;">
            My right-aligned purple text.
        </p>
        <p style="font-family: courier; color: blue; text-align: center; font-size:150%;">
            <b>My bold blue centered text in Courier font at 1.5x size.</b>
        </p>
        <h2>This was just HTML.  
    </body>
</html>

#### Now go to terminal and open the html file with your browser of choice.
For instance, on a mac:  
`$ open -a "Google Chrome" example_html_only.html`  
on ubuntu linux:  
`$ firefox example_html_only.html`

HTML is great but we don't have 6 months to teach Web Development!
------------------------------------------------------------------

So we're going to leverage our Python skills and use the Python module `Flask` to help us write and organize HTML.

Here's a simple example.  **Please read through the code and the explanatory comments thoroughly.**

In [ ]:
%%writefile flask_ex1.py

# import Flask
from flask import Flask

# initialize the Flask app, note that all routing blocks use @app
app = Flask(__name__)  # instantiate a flask app object

# routing blocks - note there is only one in this case - @app.route('/') 

# home page - the first place your app will go
@app.route('/', methods = ['GET', 'POST'])  # GET is the default, more about GET and POST below
# the function below will be executed at the host and port followed by '/' 
# the name of the function that will be executed at '/'. Its name is arbitrary.
def index():
    return 'Hello!'

# no more routing blocks

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)
    # the combination of host and port are where we look for it in our browser
    # e.g. http://0.0.0.0:8080/
    # setting debug=True to see error messages - wouldn't do this on a final deployed app

#### Now go to the terminal and type:
`$ python flask_ex1.py`  

And navigate to the specified address in the browser.  When you are done, hit Ctrl-C to stop the app and come back to this notebook.

Congratulations you just made your first app.

### Note:  the route - @app.route('/') - has two methods: GET and POST

Understanding these two terms requires putting in context how the browser (the _client_) is interacting with information stored on your local computer (the _server_). 

**GET** is used by the browser to get information from the server.  In the example above, the browser asked the server to get the information returned by the `index` function.  Every routing block has a GET method (it's the default).

**POST** tells the server that the browser wants to send information to it.  HTML _forms_ usually transmit data this way.

GET requests will show information in the browser address bar while POST won't.  Confidential information should be sent with POST.

### Let's make an app that has two pages (from two routing blocks) that we can navigate using html *forms*. 

Note that both routing blocks return strings containing HTML.  The *action* in each form navigates between the two blocks.

In [ ]:
%%writefile flask_ex2.py

from flask import Flask
app = Flask(__name__)

# first routing block
@app.route('/') # if no methods specified, default is 'GET'
def index():
    display_str = 'Hello!'
    # code below makes a button to go to the '/rainbow' block
    go_to_rainbow_html = '''
        <form action="/rainbow" >
            <input type="submit" value = "Go to rainbow"/>
        </form>
    '''
    # html that gets returned
    return display_str + go_to_rainbow_html


# second routing block
@app.route('/rainbow') # if no methods specified, default is 'GET'
def rainbow():
    colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
    color_divs = []
    for i in range(20):
        color = colors[i % len(colors)]
        # making html string that makes colors
        div = '''<div style="background-color: {0};
                             color: white;
                             text-align: center;">
                     {1}
                 </div>'''.format(color, '*' * 100)
        color_divs.append(div)
    
    # make html that gives us a button to go back to the home page
    go_to_home_html = '''
        <form action="/" >
            <input type="submit" value = "Go home"/>
        </form>
    '''
    # combine them for final html
    return '''
            <!DOCTYPE html>
            <html>
                <head>
                    <meta charset="utf-8">
                    <title>Rainbow</title>
                </head>
                <body>{0}</body>
            </html>
            '''.format('\n'.join(color_divs)) + go_to_home_html

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

#### Now go to the terminal and type:
`$ python flask_ex2.py`  

And navigate to the specified address in the browser.  When you are done, hit Ctrl-C to stop the app and come back to this notebook.

### The example above used html forms to jump back and forth between pages.

Let's use forms again and **send data** between the pages, too.

In [ ]:
%%writefile flask_ex3.py

from collections import Counter
from flask import Flask, request
app = Flask(__name__)

# formatting dictionary keys and values to be displayed in html
def dict_to_html(d):
    return '<br>'.join('{0}: {1}'.format(k, d[k]) for k in sorted(d))


# Form page to submit text
@app.route('/')
def submission_page():
    # in this form, method = 'POST' means that data entered into
    # the 'user_input' variable will be sent to the /word_counter routing
    # block when the 'Enter text' submit button is pressed
    return '''
        <form action="/word_counter" method='POST' >
            <input type="text" name="user_input" />
            <input type="submit" value = 'Enter text'/>
        </form>
        '''


# My word counter app
# this routing block accepts data in variable 'user_input' from the form
# and then processes it on the server and returns word counts to the browser
@app.route('/word_counter', methods=['GET', 'POST'])
def word_counter():
    text = str(request.form['user_input']) #gets the value entered in the input type="text", name="user_input"
    word_counts = Counter(text.lower().split())
    
    # formatted output string
    page = 'There are {0} words.<br><br>Individual word counts:<br> {1}'
    
    # make html that gives us a button to go back to the home page
    go_to_home_html = '''
        <form action="/" >
            <input type="submit" value = "Enter more text"/>
        </form>
    '''
    
    return page.format(len(word_counts), dict_to_html(word_counts)) + go_to_home_html


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

#### Now go to the terminal and type:
`$ python flask_ex3.py`  

And navigate to the specified address in the browser.  When you are done, hit Ctrl-C to stop the app and come back to this notebook.

###  Let's make a plot with matplotlib and display it.

In [ ]:
%%writefile flask_ex4.py

from random import random
import matplotlib
from matplotlib.figure import Figure
matplotlib.use('Agg')
from io import BytesIO
from flask import Flask
app = Flask(__name__)

@app.route('/')
def index():
    # img in the html specifies the presence of an image
    # however, the src of the image is a routing block instead of
    # a .jpg or .png
    return '''
        <h2>Here's my plot!</h2>
        <img src='/plot.png'>
        '''

@app.route('/plot.png')
def get_graph():
    fig = Figure()
    ax = fig.subplots()
    n = 10
    ax.plot(range(n), [random() for i in range(n)])
    image = BytesIO()
    fig.savefig(image)  #the plot is saved
    return image.getvalue(), 200, {'Content-Type': 'image/png'}


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

#### Now go to the terminal and type:
`$ python flask_ex4.py`  

### So you are probably getting tired of looking at HTML in your Python code.

You can create HTML templates and render them using Jinja.  Jinja (Jinja2) creates HTML files that optionally containing user values.  [Here](https://realpython.com/blog/python/primer-on-jinja-templating/) is a nice description of Jinja templates.
**The HTML template needs to be in a `/templates` folder in the same directory as the flask app.**  

Normally you will find example templates similar to what you want, place them in the templates folder, and then modify them to display what you want.  In this case, the "table.html" file is created by the cell below.

In [ ]:
%%writefile templates/table.html

<html>
<head>
<!-- CSS style -->
<style>
table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
}
th, td {
    padding: 15px;
}
table.center {
    margin-left:auto; 
    margin-right:auto;
}
</style>
</head>
<body>
<table class="center">
  <thead>
    <th>x</th>
    <th>y</th>
  </thead>
  <tbody>
    <!-- Here is where the data passed into the template gets inserted -->
    {% for x, y in data %}
      <tr>
        <td>{{ x }}</td>
        <td>{{ y }}</td>
      </tr>
    {% endfor %}
  </tbody>
</table>

#### Now the flask app below renders this html template, and passes data to it to be displayed.

In [ ]:
%%writefile flask_ex5.py

from flask import Flask, render_template
from random import random
app = Flask(__name__)


@app.route('/')
def index():
    n = 10
    x = range(n)
    y = [round(random(),3) for i in x]
    # table.html is an existing file in the templates folder
    # isn't this nice, no html here!
    return render_template('table.html', data=zip(x, y))  # using Jinja here, the render_template call

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

#### Now go to the terminal and type:
`$ python flask_ex5.py`  

### Now let's deploy a trained machine learning model in an app.

In this case we will:  
1. Train the model in a python file and `pickle` the fitted model.
2. Put the pickled model on the server with the web app.
3. In the flask python file, unpickle the model and use it to predict.

See the example below.

In [ ]:
# Here we train and pickle a logistic regression model on the iris dataset

# Based on:
# http://scikit-learn.org/stable/auto_examples/linear_model/plot_iris_logistic.html

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import pickle

# import data
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

# declare model
logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

# fit the data
logreg.fit(X, Y)

# pickle the data (wouldn't normally do this but want to plot prediction vs the train data)
trainXY = np.hstack((X,Y.reshape(-1,1)))
with open('data/trainXY.pkl', 'wb') as f:
    pickle.dump(trainXY, f)

# pickle the model
with open('data/model_logreg.pkl', 'wb') as f:
    pickle.dump(logreg, f)

### Now that the model has been pickled, let's use it in an app to predict on new data:

In [ ]:
%%writefile flask_ex6.py

from flask import Flask, request
import pickle
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO

# Initialize app
app = Flask(__name__)

# load the pickled model
with open('data/model_logreg.pkl', 'rb') as f:
    model = pickle.load(f)

# load the pickled training data to display with prediction
with open('data/trainXY.pkl', 'rb') as f:
    trainXY = pickle.load(f)

trainX = trainXY[:,:2]
trainY = trainXY[:,2]

# Home page with form on it to submit new data
@app.route('/')
def get_new_data():
    return '''
        <form action="/predict" method='POST'>
          Sepal length (4.0 - 9.0):<br>
          <input type="text" name="length"> 
          <br>
          Sepal width (1.0 - 5.0):<br>
          <input type="text" name="width"> 
          <br><br>
          <input type="submit" value="Submit for class prediction">
        </form>
        '''

@app.route('/predict', methods = ["GET", "POST"])
def predict():
    # request the text from the form 
    length = float(request.form['length'])
    width = float(request.form['width'])
    X_n = np.array([[length, width]])
    
    # predict on the new data
    Y_pred = model.predict(X_n)

    # for plotting 
    X_0 = trainX[trainY == 0] # class 0
    X_1 = trainX[trainY == 1] # class 1
    X_2 = trainX[trainY == 2] # class 2
    
    # color-coding prediction 
    if Y_pred[0] == 0:
        cp = 'b'
    elif Y_pred[0] == 1:
        cp = 'r'
    else:
        cp = 'g'

    if plt:
        plt.clf() # clears the figure when browser back arrow used to enter new data

    plt.scatter(X_0[:, 0], X_0[:, 1], c='b', edgecolors='k', label = 'class 0')
    plt.scatter(X_1[:, 0], X_1[:, 1], c='r', edgecolors='k', label = 'class 1')
    plt.scatter(X_2[:, 0], X_2[:, 1], c='g', edgecolors='k', label = 'class 2')
    plt.scatter(X_n[:, 0], X_n[:, 1], c=cp, edgecolors='k', marker = 'd', \
        s=100, label = 'prediction')
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.title('Prediction plotted with training data')
    plt.legend()
        
    image = BytesIO()
    plt.savefig(image)
    out = image.getvalue(), 200, {'Content-Type': 'image/png'}
    return out

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

#### Now go to the terminal and type:
`$ python flask_ex6.py`  

### You are now ready for the individual assignment!  Go to it!

The individual assignment asks you to use html templates for the index, predict and submit pages.  The `table.html` in this notebook is one example.  If you get stuck making these templates, say something and the instructor will release them.  They are nothing fancy.